<a href="https://colab.research.google.com/github/Blackbirdf16/Cryptography/blob/main/Hash.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Hash functions and random numbers


## Install necessary libraries

### Subtask:
Install the `cryptography` library and any other required libraries.


In [ ]:
%pip install cryptography numpy matplotlib

### Using hash functions in Python with Cryptography:



Demonstrate the usage of hash functions from the `cryptography` library by importing necessary modules, creating a hash object, updating it with data, finalizing the computation, and printing the results.



In [ ]:
from cryptography.hazmat.primitives import hashes

# Create a Hash object for SHA-256
digest = hashes.Hash(hashes.SHA256())

# Update the hash object with sample data
data = b"This is some sample data for hashing."
digest.update(data)

# Finalize the hash computation and get the digest
hash_digest = digest.finalize()

# Print the original data and the resulting hash digest
print(f"Original data: {data}")
print(f"SHA-256 hash digest: {hash_digest.hex()}")

Original data: b'This is some sample data for hashing.'
SHA-256 hash digest: f2bd24225dec46c10b9699a28394766588b8b1061decd4679cff7abec287bb06


## Measuring pseudorandomness


We will compute hash digests to original and modified data to demonstrate the avalanche effect by comparing the resulting hashes bit by bit.



In [ ]:
from cryptography.hazmat.primitives import hashes

def compute_hash_hex(data, algorithm):
  """Computes the hash of data using the specified algorithm and returns it as a hex string."""
  digest = hashes.Hash(algorithm)
  digest.update(data)
  return digest.finalize().hex()

def count_differing_bits(hex_str1, hex_str2):
    """Counts the number of differing bits between two hexadecimal strings."""
    int1 = int(hex_str1, 16)
    int2 = int(hex_str2, 16)

    # XOR the integers to find differing bits
    xor_result = int1 ^ int2

    # Count the number of set bits (which correspond to differing bits)
    count = 0
    while xor_result > 0:
        xor_result &= (xor_result - 1) # Brian Kernighan's algorithm to count set bits
        count += 1
    return count

# Sample data
original_data = b"This is some sample data for demonstrating the avalanche effect."

# Create modified data by flipping a single bit (e.g., the first bit)
# We'll flip the first bit of the first byte
modified_data = bytearray(original_data)
modified_data[0] ^= 1  # Flip the first bit of the first byte
modified_data = bytes(modified_data)


# Compute hashes
original_hash = compute_hash_hex(original_data, hashes.SHA256())
modified_hash = compute_hash_hex(modified_data, hashes.SHA256())

# Count differing bits
differing_bits = count_differing_bits(original_hash, modified_hash)

# Print results
print(f"Original data: {original_data}")
print(f"Modified data: {modified_data}")
print(f"Original hash (SHA256): {original_hash}")
print(f"Modified hash (SHA256): {modified_hash}")
print(f"Number of differing bits: {differing_bits}")

Original data: b'This is soe sae data for demonstrating the avalanche effect.'
Modified data: b'Uhis is soe sae data for demonstrating the avalanche effect.'
Original hash (SHA256): d485b7dd7d45b15bc040792d773349b3d1f22736f422fbaa6f6b2c71baba21bc
Modified hash (SHA256): bfdcba67a104eea6a8d3bf90680b97c360f95685ff92bdc11c8c19a863ea64b0
Number of differing bits: 133


## Exercises

1. Can you compute the average difference repating the experiment N times?
2. How many times the first bit changes?

## Csprng

Implement a cryptographically secure pseudorandom number generator using a secure hash function (SHA-256).


The `InsecureCSPRNG` class demonstrates a simplified, and importantly, **insecure** method of generating pseudorandom numbers using SHA-256. It is designed to highlight the importance of a proper state update mechanism in a cryptographically secure generator.

**How this (Insecure) Generator Works:**

1.  **Initialization:** It starts with a `seed`, similar to a secure CSPRNG.
2.  **State:** It maintains an internal `_state`, initialized with the seed.
3.  **Generation:** When `generate(num_bytes)` is called, it repeatedly hashes the *current* `_state` to produce blocks of output bytes.
4.  **Insecure State Update:** The critical vulnerability lies in how the `_state` is updated. After generating a block of output, the `_state` is simply updated by hashing the *previous* state (`self._state = digest_update.finalize()`).

**Why this is Insecure?**


In [ ]:
from cryptography.hazmat.primitives import hashes
import os

class InsecureCSPRNG:
    """
    An intentionally insecure Cryptographically Secure Pseudorandom Number Generator (CSPRNG)
    using a simple state update mechanism for demonstration purposes.
    """
    def __init__(self, seed):
        """
        Initializes the CSPRNG with a seed.

        Args:
            seed (bytes): The initial seed for the CSPRNG.
        """
        if not isinstance(seed, bytes):
            raise TypeError("Seed must be bytes.")
        self._state = seed
        self._hash_size = hashes.SHA256().digest_size # Get the output size of the hash function

    def generate(self, num_bytes):
        """
        Generates a specified number of pseudorandom bytes.

        Args:
            num_bytes (int): The desired number of random bytes to generate.

        Returns:
            bytes: The generated pseudorandom bytes.
        """
        generated_bytes = b""
        while len(generated_bytes) < num_bytes:
            # Generate the next block of random bytes by hashing the current state
            digest = hashes.Hash(hashes.SHA256())
            digest.update(self._state)
            hash_output = digest.finalize()

            generated_bytes += hash_output

            # *** INSECURE STATE UPDATE MECHANISM ***
            # The state is updated simply by hashing the previous state.
            # This is vulnerable to attacks where an attacker can predict future states
            # if they know the current state or output.
            digest_update = hashes.Hash(hashes.SHA256())
            digest_update.update(self._state)
            self._state = digest_update.finalize()
            # *************************************

        return generated_bytes[:num_bytes]

# Example usage:
# Create an instance of the insecure CSPRNG
insecure_seed = os.urandom(32) # Using os.urandom for initial unpredictability
insecure_rng = InsecureCSPRNG(insecure_seed)

# Generate some random data on demand
random_data_1 = insecure_rng.generate(16)
print(f"Generated random data 1 ({len(random_data_1)} bytes): {random_data_1.hex()}")

random_data_2 = insecure_rng.generate(32)
print(f"Generated random data 2 ({len(random_data_2)} bytes): {random_data_2.hex()}")

Generated random data 1 (16 bytes): 116b7357706c1ac623f8e7e875f7a86c
Generated random data 2 (32 bytes): e72ac4d226501ea5c5fb6d36caff2c166709a8541a83d49730a358e99ef79b7b


## Exercise


1.   Fix the code to make it secure.



## Estimate pi using monte carlo simulation

Use the implemented CSPRNG to perform a Monte Carlo simulation to estimate the value of pi.


In [ ]:
import math

def estimate_pi_monte_carlo(seed, num_points):
    """
    Estimates the value of pi using a Monte Carlo simulation with a CSPRNG.

    Args:
        seed (bytes): The seed for the CSPRNG.
        num_points (int): The total number of points to generate for the simulation.

    Returns:
        float: The estimated value of pi.
    """
    # We need 2 random numbers (x and y) for each point
    # Each random number will be represented by a float between 0 and 1
    # A float typically takes 8 bytes, so we need num_points * 2 * 8 bytes
    # However, generating bytes and then converting to float is more robust
    # Let's generate enough bytes to get num_points * 2 random integers,
    # and then scale them to be between 0 and 1.
    # We'll use 4 bytes per random integer for sufficient resolution.
    bytes_needed = num_points * 2 * 4 # 4 bytes per integer

    # Use the InsecureCSPRNG class instance
    rng = InsecureCSPRNG(seed)
    random_bytes = rng.generate(bytes_needed)


    # Convert bytes to integers and then to floats between 0 and 1
    random_numbers = []
    # The maximum value for a 4-byte integer is 2^32 - 1
    max_int_value = 2**32 - 1
    for i in range(0, bytes_needed, 4):
        # Convert 4 bytes to an integer
        random_int = int.from_bytes(random_bytes[i:i+4], 'big')
        # Scale the integer to a float between 0 and 1
        random_float = random_int / max_int_value
        random_numbers.append(random_float)

    # Count points inside the circle
    points_inside_circle = 0
    for i in range(0, len(random_numbers), 2):
        x = random_numbers[i]
        y = random_numbers[i+1]
        # Check if the point (x, y) is within the unit circle's top-right quadrant
        if x**2 + y**2 <= 1:
            points_inside_circle += 1

    # Estimate pi
    estimated_pi = 4 * points_inside_circle / num_points

    return estimated_pi

# Set a seed and the number of points
simulation_seed = os.urandom(32) # Use a new random seed for the simulation
num_simulation_points = 100000

# Run the simulation and print the result
estimated_pi_value = estimate_pi_monte_carlo(simulation_seed, num_simulation_points)
print(f"Estimated value of pi using Monte Carlo simulation with {num_simulation_points} points: {estimated_pi_value}")
print(f"Actual value of pi: {math.pi}")

Estimated value of pi using Monte Carlo simulation with 100000 points: 3.13784
Actual value of pi: 3.141592653589793


## "Pre-image" resistance



1.   Make a program to find strings like this:

```
"The sha256sum value of this string ends with a27e"
```

2.   Make a program to find strings like this:

```
"The SHA-256 hash of this sentence begins with 0573e7473."
```

